In [1]:
from FME.interpolators.piecewiselinear_interpolator import PiecewiseLinearInterpolator as PLI
from FME.supports.tet_mesh import TetMesh
from FME.modelling.features.geological_feature import GeologicalFeatureInterpolator
# from FME.modelling.features.faulted_geological_feature import FaultedGeologicalFeature
from FME.visualisation.model_visualisation import LavaVuModelViewer
# from FME.modelling.structural_frame import StructuralFrameBuilder, StructuralFrame
# from FME.modelling.fault.fault_segment import FaultSegment
import numpy as np
import lavavu
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Implicit benchmark data set - Moureze, synthetic hydrothermal body. 

*Data set interpreted and prepared in July 2019 by Guillaume.Caumon@univ-lorraine.fr, 
to whom questions should be addressed.*

This data set is generated from a perturbed distance field to a set of fractures
above a basement surface. The surface is quite complex and has many features at
two main resolutions (one corresponding to the fractures and the other one to
the noise). It is samples by two sets of cross section lines and a set of
points. 

## Contents
 
 * `Moureze_Points.csv` correspond are samples of the surface to reconstruct. 
 Their density decreases from south to north (the surface is clearly
 undersampled in the north). Some of the points bear
 orientation vectors representing the normal to the surface. 
 * `Sections_[EW,NS].csv` are sections lines. The field `PART_ID` is a unique 
 section identifier in each file. 

## Results

Please rename the `Results_` as `Results_YOURNAME` directory. The directory
contains an empty Voxet (Cartesian Grid) and an empty 
tetrahedral mesh conformable to faults (resolution 3m), both 
in Gocad ASCII format.

### Scalar Field
Suggested size of the axis-aligned modeling box: 
Origin: -5 -5 -200
Maximum: 305 405 -50

Suggested resolution: 2m (grid size 156 x 206 x 76)

An empty Gocad Voxet is already in the results directory. You may write your values in binary format (float C 4-byte values), or ASCII format.  

### Surfaces 

Please use DXF, Gocad and/or .obj format. 


In [2]:
dips = pd.read_csv('../Moureze_Points.csv',delimiter=';')
# points_file_names = ['../Moureze_Points.csv']
section_file_names = ['../Sections_EW.csv','../Sections_NS.csv']

# import all of the csv into the same dataframe
# dfs = []
# for f in points_file_names:
#     dfs.append(pd.read_csv(f,delimiter=';'))
# points = pd.concat(dfs,axis=0,ignore_index=True)
# sections
dfs = []
for f in section_file_names:
    dfs.append(pd.read_csv(f,delimiter=';'))
sections = pd.concat(dfs,axis=0,ignore_index=True)
# apoints = pd.read_csv('APoints.csv',delimiter=';')
# asection = pd.read_csv('ASection.csv',delimiter=';')



In [3]:
boundary_points = np.zeros((2,3))

boundary_points[0,0] = -5
boundary_points[0,1] = -5
boundary_points[0,2] = -200
boundary_points[1,0] = 305
boundary_points[1,1] = 405
boundary_points[1,2] = -50
mesh = TetMesh(name='Moureze')
mesh.setup_mesh(boundary_points, nstep=1, n_tetra=100000,)


In [4]:
print(sections)

                X           Y        Z  PART_ID  Unnamed: 4
0      298.000000  372.000000 -158.251        1         NaN
1      296.226807  372.000000 -158.147        1         NaN
2      296.136200  372.000000 -158.136        1         NaN
3      296.000000  372.000000 -158.120        1         NaN
4      295.487976  372.000000 -158.000        1         NaN
5      294.000000  372.000000 -157.652        1         NaN
6      293.523773  372.000000 -157.524        1         NaN
7      292.000000  372.000000 -157.113        1         NaN
8      290.739380  372.000000 -156.739        1         NaN
9      290.000000  372.000000 -156.520        1         NaN
10     288.058472  372.000000 -156.000        1         NaN
11     288.000000  372.000000 -155.984        1         NaN
12     287.981232  372.000000 -155.981        1         NaN
13     286.000000  372.000000 -155.655        1         NaN
14     285.623291  372.000000 -155.623        1         NaN
15     284.000000  372.000000 -155.484  

In [5]:
interpolator = PLI(mesh)
stratigraphy_builder = GeologicalFeatureInterpolator(
    interpolator=interpolator,
    name='stratigraphy')
solver = 'lu'
# for i, r in points.iterrows():
#     stratigraphy_builder.add_point([r['X'],r['Y'],r['Z']],r['Strati'])#xy[0][0],xy[1][0],z],r['value'],itype=r['itype'])
for i, r in sections.iterrows():
    stratigraphy_builder.add_point([r['X'],r['Y'],r['Z']],r['PART_ID'])#xy[0][0],xy[1][0],z],r['value'],itype=r['itype'])
for i, r in dips.iterrows():
    stratigraphy_builder.add_planar_constraint([r['X'],r['Y'],r['Z']],[r['OrientX'],r['OrientY'],r['OrientZ']])
# stratigraphy_builder.add_point([6.1,0.1,1.1],0.)
# stratigraphy_builder.add_point([6.1,0.1,2.1],1.)
# stratigraphy_builder.add_strike_and_dip([1,1,1],90.,0.)
unique = np.unique(stratigraphy_builder.interpolator.get_control_points()[:,3])

In [6]:
stratigraphy = stratigraphy_builder.build(solver=solver,cgw=100)


In [ ]:
viewer = LavaVuModelViewer(background="white")
viewer.plot_isosurface(stratigraphy, slices=unique,  colour='blue')
viewer.plot_value_data(stratigraphy.support.interpolator.get_control_points()[:,:3],
                       stratigraphy.support.interpolator.get_control_points()[:,3],
                       'val')
viewer.plot_isosurface(stratigraphy, slices=[30,125,300], colour='red')
viewer.plot_vector_data(
    stratigraphy.support.interpolator.get_gradient_control()[:,:3],
    stratigraphy.support.interpolator.get_gradient_control()[:,3:],
    'grad')

viewer.interactive()

In [8]:
viewer.lv.image("~/dev/python/FME/examples/Claudius/loop.png")

'~/dev/python/FME/examples/Claudius/loop.png'